# Generisanje teksta

Primer rekurentne neuronske mreže (RNN) za generisanje teksta na osnovu datog korpusa. 
Moguć izbor generisanja:
* teksta na engleskom jeziku
* teksta pesme na srpskom jeziku za željenog izvođača.

Potrebno je oko 20ak epoha da generisani tekst zvuči koherentno.
Poželjno je da se ovo izvršava na GPU.
Ako se ovo primenjuje na novim podacima, poželjno je da isti imaju oko 100k karaktera, idealno blizu 1M.

In [ ]:
import io
import os
import sys
import random

import numpy as np

from tensorflow import keras
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM, Input
from keras.optimizers import RMSprop
from keras.utils import get_file

## Korpus

### Izbor

In [ ]:
POSSIBILITES = [
    # english
    'nietzsche',
    'crusoe',
    # serbian
    'folk',
    'rock',
    'pop'
]

MODEL_MODE = POSSIBILITES[2]
ARTIST = "Mitar"

### Priprema

In [ ]:
if MODEL_MODE == 'nietzsche':
    # preuzimanje teksta 
    path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
    with io.open(path, encoding='utf-8') as f:
        text = f.read().lower()

elif MODEL_MODE == 'crusoe':
    # učitavanje teksta iz fajla
    with open('./data/crusoe.txt', 'r') as crusoe_file:
        text = crusoe_file.read()

elif MODEL_MODE in ['folk', 'pop', 'rock']:
    # učitavanje svih tekstova za željenog izvođača
    text = ""
    file_names = [f for f in os.listdir('./data/' + MODEL_MODE + '/') if os.path.isfile(os.path.join('./data/' + MODEL_MODE + '/', f))]
    for file_name in file_names:
        if not file_name.startswith(ARTIST):
            continue
        with open('./data/' + MODEL_MODE + '/' + file_name, 'r') as song_text_file:
            text += song_text_file.read() + '@\n' # generisaće tekst dok ne dođe do @
    text = text.replace('"', '')

In [ ]:
print("Dužina korpusa:", len(text))

chars = sorted(list(set(text)))
print("Ukupno jedinstvenih karaktera:", len(chars))
print(chars)

In [ ]:
# indeksiranje karaktera
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

## Pretprocesiranje

Isecanje teksta u sekvence `maxlen` karaktera

In [ ]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print("Broj sekvenci:", len(sentences))

## Vektorizacija

In [ ]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

print("Dimenzije ulaza:", x.shape)
print("Dimenzije izlaza:", y.shape)

## Model

### Arhitektura

In [ ]:
model = Sequential()
model.add(Input(shape=(maxlen, len(chars))))
model.add(LSTM(128))
model.add(Dense(len(chars), activation='softmax'))

model.summary()

### Optimizacija

In [ ]:
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

## Treniranje

Pomoćna funkcija za uzorkovanje indeksa iz niza verovatnoća:

In [ ]:
def sample(preds, temperature=1.0):
    # temperature parametar za "inovativnost" mreže - omogućava slovima koja i nisu najverovatnija da budu izabrana
    # 0 - manje eksperimentiše/inovira, 1 - više eksperimentiše/inovira
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Funkcija koja se poziva na kraju svake epohe kako bi prikazala generisani tekst:

In [ ]:
def on_epoch_end(epoch, _):
    print()
    print('\n----- Generisanje teksta nakon epohe: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('\n----- Inovativnost (temperatura):', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generisanje na osnovu: "' + sentence + '"')
        print('----- Generisani tekst:')
        sys.stdout.write(generated)

        for i in range(400): 
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            if next_char == '@':
                break

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [ ]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

Treniranje i ispis teksta nakon svake epohe:

In [ ]:
model.fit(x, y, batch_size=1024, epochs=60, callbacks=[print_callback])